## Transformer
理论解释来自 http://jalammar.github.io/illustrated-transformer/ ， 代码来自 https://pytorch.org/tutorials/beginner/transformer_tutorial.html 。

Transformer由6个（个数可任意选取）encoder和decoder组成。
### 一、encoder
6个encoder的结构相同、参数不同，其由两个layers构成：
1. Self-Attention layer，该层使编码器在对某个单词进行编码时也能够记录其他单词的信息，即并行计算——缓解了RNN串行计算的缺点；
2. Feed-Forward network，自注意力层的输出结果将被传递到前馈神经网络中。
decoder除了包含Self-Attention layer和FFN，还包括Encoder-Decoder Attention layer，凭借该层解码器既能够关注本身的输入信息，也能够获取encoder所传入的信息。

#### （一）自注意力机制
如何使计算机在处理某个单词时能够“注意到”其他单词？自注意力机制提供了一个解决方案。如“我今天吃饭了吗”，主语“我”对应动词“吃”，故“我”与“吃”的词向量相似度较高，即在处理“吃”时，应更多地关注“我”。自注意力机制的核心思想是为每个单词分配一个权重，该权重通过向量的相似度（内积）来实现，在以该权重来计算所有单词对某个单词的最终输出的影响。
步骤：
1. 将每个单词转换为维数相同的词向量$x$，词向量的维数属于超参数，论文中给定的维数是512。注意：只有最底层的encoder，即第一个encoder才会接收词向量，而对于其他的encoders，则接收上一层encoder的输出值作为输入值。
2. 设置三个矩阵$w_q、w_k、w_v$，并分别与词向量$x$做内积，得到向量$q、k、v$，分别表示“查询”、“键”、“值”；
3. 再将$q、k$做内积，即得到查询与各个键之间的相似度，即某个单词与所有单词的相似度，之后再除以向量的维度（确保方差为1，使梯度向量更稳定），以及施加softmax函数，目的是得到各单词被选择的概率；
4. 最终得到：
$$Attention(Q,K,V)=softmax\left(\frac{QK^T}{\sqrt {d_k}}\right)V$$
5. 实际计算中是采用基于矩阵的计算方式，对于每个$x$，均存在与之相对应的$q、k、v$，将所有的$x$组合成$X$，则$q、k、v$也组合为$Q、K、V$；假设$x$维度为1×512，共有100个单词，$w_q、w_k、w_v$是512×64，则：$$Attention(Q,K,V)=softmax\left(\frac{100×64\quad dot \quad64×100}{\sqrt {d_k}}\right)100×64$$故经过self-attention层后，词向量的维度将被替换为$w_q、w_k、w_v$的维度。

在原论文中，作者所采用的Multi-head attention layers由多个$Q、K、V$组成，故对于每个单词存在多个输出，将多个输出横向拼接，再与矩阵$w_o$相乘，即可得到最终的输出结果。多头注意力层的作用是，每个attention head均能关注不同的单词，从而捕捉更丰富的特征信息。
#### （二）残差网络
在self-attention和FFN之间加入了层标准化后的残差网络，即$$LayerNorm(X+Z)$$引入残差网络的目的是解决深度学习中的退化问题，缓解梯度消失，避免长期遗忘，使得信息能够从低层传递到高层中。

输入向量A，经过函数B和C，再加上A本身，得到输出D，即：$$D=C[B(A)]+A$$
损失函数$L$对A求偏导可得：
$$\frac{\partial L}{\partial A}=\frac{\partial L}{\partial D}\frac{\partial D}{\partial A}=\frac{\partial L}{\partial D}\left(\frac{\partial D}{\partial C}\frac{\partial C}{\partial B}\frac{\partial B}{\partial A}+1\right)$$
观察梯度向量的表达式可得，低层前往高层的梯度向量为0的可能性降低了，即梯度消失的可能性减小了，从而解决了深层神经网络退化的问题。

#### （三）位置编码
为了使transformer具备捕捉位置信息的能力，论文在编码词向量的过程中引入了位置编码，对于偶数位置使用sin函数，对于奇数位置采用cos函数，再将三角函数值与词向量对位相加。使用三角函数的好处是，由于$sin(p+k)=sinpcosk+sinkcosp$，故三角函数可将位置信息分解为绝对位置信息和相对位置信息，同时此种位置编码方法能够识别测试集向量中序列长度大于训练集序列长度的向量。位置编码的公式为：
$$PE(pos,2i)=sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$
$$PE(pos,2i+1)=cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

### 二、Decoder
#### （一）Encoder-Decoder Attention
1. 最高层的encoder的输出值与矩阵相乘后得到注意力向量$K、V$，$K、V$被decoders在其encoder-decoder attention层中使用，使decoder能够关注输入信息中正确的信息，而$Q$矩阵则由下方的decoder传入；
2. 最高层decoder的输出结果被返回到最底层的decoder，作为输入。以翻译任务为例，假设将中文翻译为英文，则encoder的输入为中文，而decoder的输入为英文
3. 为了符合因果关系，decoder在预测单词时只允许关注该位置之前的单词，故该位置之后的单词被masked了，即将数值设为负无穷，之后在softmax层中转换为接近0的数。
#### （二）Linear层与Softmax层
linear层将decoders的输出向量投影到一个更高维的空间，投影后的向量也称为逻辑向量，这是因为decoders的输出向量为词向量，只有512维，而一门语言的常用单词数目绝对不止512个单词，故需将词向量进行扩维，从而满足要求。
2. 输出的词向量扩维之后，再经过softmax函数，即可得到每个单词的概率。
#### （三）损失函数
1. 无监督学习：以softmax的输出向量中值最大的分量所对应的单词的one-hot编码，以及输出向量本身作为自变量传入损失函数中，即可求得损失值，之后再根据反向传播算法对权重进行调节
2. 监督学习：需要人工标注的数据，如翻译任务中，将“我”翻译为“I”，则需要事先标注“我”的正确输出向量为“I”的one-hot编码，再将该one-hot编码与实际输出向量计算损失值。
### 三、代码

In [23]:
import torch
from torch import nn
from torch.utils.data import dataset

class Transformer(nn.Module):
    def __init__(self,n_token,d_model,n_head,n_layers,d_FNN,dropout=0.5):#n_token是常用单词的个数，d_model表示词向量的长度
        super().__init__()

        self.d_model=d_model#生成词向量时使用
        self.model_name='transformer'

        #encoder
        self.embedding=nn.Embedding(n_token,d_model)#Embedding方法的第一个参数是词向量的个数，第二个参数是词向量的维度
        self.pos_encoder=nn.PositionalEncoder(d_model,dropout)#位置编码需自己定义
        encoder_layer=nn.TransformerEncoderLayer(d_model,n_head,d_FNN,dropout)#TransformerEncoderLayer的参数分别是d_model、n_head、FFN的维度以及dropout
        self.encoder=nn.TransformerEncoder(encoder_layer,n_layers)#TransformerEncoder方法的第一个参数是实例化的TransformerEncoderLayer，第二个参数是该实例的个数

        #decoder
        self.decoder=nn.Linear(d_model,n_token)

        #initialization
        self.init_weights()
    
    def initWeights(self):
        initrange=0.1

        self.embedding.weight.data.uniform_(initrange,initrange)#Embedding类拥有weight属性，由于weight属性包含了梯度，故需调用data提取出权重，最后使其服从均匀分布

        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(initrange,initrange)

    def forward(self,src,src_mask):#src代表输入数据
        src=self.embedding(src)#第一步：生成词向量
        src *= self.d_model**0.5

        src=self.pos_encoder(src)#第二步：词向量与位置编码对位相加
        output=self.encoder(src)#第三步：传入encoders并返回output
        output=self.decoder(output)#第四步：传入decoders并返回output
    
        return output

In [56]:
import math

def genUpperTriMatrix(n):#返回右上三角矩阵，用于mask
    return torch.triu(torch.ones(n,n)*float('-inf'),diagonal=1)

class positionalEncoder(nn.Module):
    def __init__(self,d_model,max_len=5000,dropout=0.1):#max_len代表单词个数
        super().__init__()
        self.dropout=nn.Dropout(dropout)
        
        pos=torch.arange(max_len).unsqueeze(1)
        div_term=torch.exp(-torch.arange(0,d_model,2)/d_model*math.log(10000))

        pe=torch.zeros(max_len,1,d_model)
        pe[:,0,0::2]=torch.sin(pos*div_term)#0::2表示从序列起始位置加0，步长为2
        pe[:,0,1::2]=torch.cos(pos*div_term)

        self.register_buffer('pe', pe)
    
    def forward(self,x):
        #x: Tensor, shape [seq_len, batch_size, embedding_dim]
        x+=self.pe[:x.size(0)]#x的第0维的size表示单词的个数
        return self.dropout(x)

In [61]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [72]:
train_iter=WikiText2(split='train')
tokenizer=get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

def data_process(raw_text_iter: dataset.IterableDataset):
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

train_iter,val_iter,test_iter=WikiText2()
train= data_process(train_iter)
val = data_process(val_iter)
test= data_process(test_iter)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batchify(data,bsz):
    batch_seq_len = data.size(0) // bsz
    data = data[:batch_seq_len * bsz]
    data = data.view(bsz, batch_seq_len).t().contiguous()#view就是reshape
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train, batch_size)  # shape [seq_len, batch_size]
val_data = batchify(val, eval_batch_size)
test_data = batchify(test, eval_batch_size)